In [12]:
import requests
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display
import json

m_user = "tester"
m_passw = ""
m_server = "https://"

def single_result_get(padevice, paResult): 
    response_val = ""
    try: 
        if len(padevice)>14: 
            # mac
            response = requests.get(m_server+'dut'+padevice+'/'+paResult, auth=(m_user, m_passw))
            response_val = response.json()['value']
        elif len(padevice)==14: 
            # nfc
            response = requests.get(m_server+'lookupduts/NFCT_TAG/'+padevice, auth=(m_user, m_passw))
            response = requests.get(m_server+'dut'+response.json()[0]+'/'+paResult, auth=(m_user, m_passw))
            response_val = response.json()['value']
        else:
            # serial
            response = requests.get(m_server+padevice+'/'+paResult, auth=(m_user, m_passw))
            response_val = response.json()['value']
    except: 
        response_val = ""

    return response_val

def multi_result_get(device):
    serial = single_result_get(device, 'SERIAL')
    #print(serial)
    response = requests.get(m_server+serial, auth=(m_user, m_passw))
    #print(response)
    #print(response.json())

    for element in response.json(): 
        print(f"{element['timestamp']} | {element['name']} | {element['value']}")

    return response.json()

def pairing_get(device):
    in_data = device

    hw_init = single_result_get(in_data, 'HW_VERSION')
    hw_mac = single_result_get(in_data, 'HW_ADDRESS')
    display = []
    sensor = []

    if hw_init.split('.')[2]=='0': 
        response = requests.get(m_server+'lookupduts/PAIRING1/'+hw_mac, auth=(m_user, m_passw))
        if response.json(): 
            sensor = {'mac':response.json()[0]}
        else: 
            response = requests.get(m_server+'lookupduts/PAIRING2/'+hw_mac, auth=(m_user, m_passw))
            if response.json(): 
                sensor = {'mac':response.json()[0]}
    else: 
        sensor = {'mac':hw_mac}

    sensor['serial'] = single_result_get(sensor['mac'], 'SERIAL')
    sensor['nfc'] =single_result_get(sensor['mac'], 'NFCT_TAG')

    display.append({'mac':single_result_get(sensor['mac'], 'PAIRING1')})
    display[-1]['serial'] = single_result_get(display[-1]['mac'], 'SERIAL')
    display[-1]['nfc'] = single_result_get(display[-1]['mac'], 'NFCT_TAG')

    display.append({'mac':single_result_get(sensor['mac'], 'PAIRING2')})
    display[-1]['serial'] = single_result_get(display[-1]['mac'], 'SERIAL')
    display[-1]['nfc'] = single_result_get(display[-1]['mac'], 'NFCT_TAG')

    #print(f"sensor: {sensor}")
    #print(f"display: {display}")

    return {'sensor':sensor, 'display':display}

user_input = widgets.Text(
    value=m_user, 
    #placeholder='Type something',
    description='User:',
    disabled=False
)
pass_input = widgets.Text(
    value=m_passw, 
    #placeholder='Type something',
    description='pass:',
    disabled=False
)
server_input = widgets.Text(
    value=m_server, 
    #placeholder='Type something',
    description='Serverstring',
    disabled=False
)
button_setting_apply = widgets.Button(description="Apply settings")
output_settings = widgets.Output()

h_box_settings = widgets.HBox([user_input, pass_input, server_input, button_setting_apply])
display(h_box_settings)
display(output_settings)

def on_button_setting_apply_clicked(b):
    with output_settings:
        global m_user
        global m_passw
        global m_server
        output_settings.clear_output()
        m_user = user_input.value
        m_passw = pass_input.value
        m_server = server_input.value
        #print(m_passw)

button_setting_apply.on_click(on_button_setting_apply_clicked)


device = widgets.Text(
    value='4100068', 
    description='Serial, NFC, MAC:',
    disabled=False
)
button = widgets.Button(description="Show Results")
output = widgets.Output()
h_box_result = widgets.HBox([device, button])
display(h_box_result)
display(output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        #print(m_user, m_passw, m_server)
        multi_result_get(device.value)

button.on_click(on_button_clicked)

device_pairing = widgets.Text(
    value='4100068', 
    description='Serial, NFC, MAC:',
    disabled=False
)
button_pairing = widgets.Button(description="Get Pairing")
output_pairing = widgets.Output()

h_box = widgets.HBox([device_pairing, button_pairing])
display(h_box)
display(output_pairing)

def on_button_pairing_clicked(b):
    with output_pairing:
        output_pairing.clear_output()
        print(json.dumps(pairing_get(device_pairing.value),indent = 4))
button_pairing.on_click(on_button_pairing_clicked)


Output()

Output()

Output()